In [ ]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2 gradio

In [ ]:
import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import re
import gradio as gr

In [ ]:
AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)
openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [ ]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

In [ ]:
minutes = ""

def gradioMinutizer(filename):
  global minutes
  drive.mount("/content/drive")
  audio_filename = "/content/drive/MyDrive/" + filename

  audio_file = open(audio_filename, "rb")
  transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")

  system_message = "You are an assistant that produces minutes of meetings from transcripts. The minutes have a summary, and they mention key discussion points, takeaways, and action items with their owners, in markdown."
  user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

  messages = [
      {"role": "system", "content": system_message},
      {"role": "user", "content": user_prompt}
    ]

  quant_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16,
      bnb_4bit_quant_type="nf4"
  )

  tokenizer = AutoTokenizer.from_pretrained(LLAMA)
  tokenizer.pad_token = tokenizer.eos_token
  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
  streamer = TextStreamer(tokenizer)
  model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
  outputs = model.generate(inputs, max_new_tokens=2000) #leaving out the streaming business because we need to get rid of unwanted output first

  response = tokenizer.decode(outputs[0])
  response = list(re.finditer("(?:<\|end_header_id\|>)([^<]+)(?:<\|eot_id\|>)", str(response), re.DOTALL))[-1].group(1)

  minutes = response

  return response

In [ ]:
def saveMinutes(path):
  global minutes
  drive.mount("/content/drive")
  with open("/content/drive/MyDrive/" + path, "w", encoding="utf-8") as f:
    f.write(minutes)

In [ ]:
with gr.Blocks(
        css="""
    .red-button {
        background-color: darkred !important;
        border-color: red !important;
    }
    .blue-button {
        background-color: darkblue !important;
        border-color: blue !important;
    }
    .green-button {
        background-color: green !important;
        border-color: green !important;
    }
    """
) as view:
  with gr.Row():
    title = gr.HTML("<h2>Welcome to the <big>M</big>ighty</h2><h1><big>M</big>eeting <big>M</big>inute <big>M</big>achine</h1>")
    subtitle = gr.HTML("<h3>Create meeting minutes in mere …minutes – and three easy steps:</h3><ol><li>Enter the location of the mp3 recording of the meeting on your Google Drive</li>\
<li>Specify the output file (.md) on your Google Drive (leave empty if you don't need it)</li>\
<li>Hit  <q><big>M</big>ake <big>M</big>e <big>M</big>y <big>M</big>inutes</q></div>")
  with gr.Row():
    inputPath   = gr.Textbox(label="Specify the location of the mp3 recording on your Google Drive")
  with gr.Row():
    liveMinutes = gr.Markdown(
        value='<div style="color: #999; padding: 10px;">Your meeting minutes will appear here...</div>',
        label="Meeting minutes:",
         min_height=60
        )
  with gr.Row():
    minutizer = gr.Button(value="Make Me My Minutes!", elem_classes="blue-button")
  with gr.Row():
    clear = gr.Button(value="Clear everything", elem_classes="red-button")
  with gr.Row():
    outputPath  = gr.Textbox(label="Specify the desired name and location on your Google Drive for the minutes file (.md) to be saved")
  with gr.Row():
    save  = gr.Button(value="Save My Minutes!", elem_classes="blue-button")

  minutizer.click(gradioMinutizer, inputs=inputPath, outputs=liveMinutes)
  clear.click(lambda: gr.update(value=""), [], [inputPath]
       ).then(lambda: gr.update(value='<div style="color: #999; padding: 10px;">Your meeting minutes will appear here...</div>'), [], [liveMinutes]
       ).then(lambda: gr.update(value=""), [], [outputPath]
       ).then(lambda: gr.update(value="Save My Minutes!", elem_classes="blue-button"), [], [save]
       )
  save.click(saveMinutes, inputs=outputPath, outputs=[]).then(lambda: gr.update(value="Minutes saved!", elem_classes="green-button"), [], [save])

view.launch(share=True) #, debug=True)